# Import libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pylab
%matplotlib inline

# Functions

In [ ]:
def rbe_wedenberg(dose, let, abx,q):
    """
    Wedenberg proton RBE model
    input parameters may be either numpy.array or scalars
    TODO: handle Cube() class directly
    :params dose: physical proton dose in [Gy]
    :params let: LETd in [keV/um] (protons only)
    :params abx: alpha_x / beta_x [Gy]
    :returns: RBE for the given parameters
    :ref: http://dx.doi.org/10.3109/0284186X.2012.705892
    """

    _apx = 1.000 + q * let / abx
    _sbpx = 1.0

    rbe = _rbe_apx(dose, _apx, _sbpx, abx)
    return rbe

In [ ]:
def _rbe_apx(dose, apx, sbpx, abx, dzero=0.0):
    """
    :params dose: proton dose      [Gy]
    :params apx: alpha_p / alpha_x [dimensionless] RBE_max = ap/ax when (dose -> 0 Gy)
    :params sbpx: beta_p / beta_x  [dimensionless] RBE_min = sqrt(bp/bx) when (dose -> inf Gy)
    :params abx: alpha_x / beta_x  [Gy]
    :params dzero: what to return in case of dose is zero (which would cause division by zero)
    """

    _rbe = 1.0 / (2.0 * dose)
    if hasattr(_rbe, '__iter__'):
        _rbe[_rbe == np.inf] = dzero
    else:
        if _rbe == np.inf:
            return dzero
    delta = abx * abx + 4. * abx * apx * dose + 4. * sbpx * sbpx * dose * dose
    delta *= (delta > 0)
    _rbe *= (np.sqrt(delta) - abx)
    return _rbe

In [ ]:
style_param = {'legend.fontsize': '20',
         'xtick.direction' : 'in',  
         'ytick.direction' : 'in', 
         'xtick.top' : True, 
         'figure.figsize': (10,6),
         'axes.labelsize': '26',
         'axes.titlesize':'26',
         'xtick.labelsize':'20',
         'ytick.labelsize':'20',
         'xtick.major.pad':'16',
         'ytick.major.pad':'16'}
    
pylab.rcParams.update(style_param)

# Open data

In [ ]:
open_fname = os.path.join('tmp','distrib_q.h5')
q = pd.read_hdf(open_fname, 'data_1')

In [ ]:
fname = os.path.join('data',"sobp_10mln")
dose = os.path.join(fname,'dose.dat')
let=os.path.join(fname,'dlet.dat')

In [ ]:
dozym = os.path.join('data','doz_sobp.xlsx')
df_dozym = pd.read_excel(dozym,names=['depth','dose','depth2','dose2'])

In [ ]:
df_dose=pd.read_table(dose,names=['z','dose','err'],sep='\s+')
df_let=pd.read_table(let,names=['z','let','err'],sep='\s+')

In [ ]:
# LET and normalized dose in water phantom
dose_max=df_dose.dose[df_dose.dose>0.95*df_dose.dose.max()].mean()
rbe= pd.DataFrame(dtype='float')

rbe["dose"]=df_dose['dose']*2/dose_max
rbe["let"]=df_let['let']
rbe["z"]=df_dose['z']

# Wedenberg RBE distribution for LET=2.

In [ ]:
tmp = rbe_wedenberg(2.0,3.0, 2.0, q )

In [ ]:
fig,ax = plt.subplots(figsize = (10,6))
tmp.hist(bins=(100), density=True,color='grey',ax=ax)
ax.axvline(x=1.1, label='RBE = 1.1'.format(1.1),c='k',ls='--',lw=1.2)

ax.set_title("")
ax.set_xlabel("\nRBE [-]")
ax.set_ylabel("Rozkład gęstości \nprawdopodobieństwa [-]\n")
ax.legend()

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')

In [ ]:
tmp=pd.DataFrame(list(zip(tmp)),
                          columns=['rbe'])
tmp.to_csv("results/figure_5_9")
fig.savefig(fname="results/figure_5_9",dpi= 700)

In [ ]:
#RBE for q=0.434 from Wedenberg
abx=2.
rbe['wed'] = rbe_wedenberg(rbe.dose, rbe.let, abx, 0.434)
rbe['wed_q_down'] = rbe_wedenberg(rbe.dose, rbe.let, abx, 0.366)
rbe['wed_q_up'] = rbe_wedenberg(rbe.dose, rbe.let, abx, 0.513)

# our RBE distribution

In [ ]:
dfr = (rbe.reset_index(inplace=False))

In [ ]:
#optional : n=q.size-1
n=1000-1

In [ ]:
dfr =dfr.append([dfr]*n)

In [ ]:
dfr=(dfr.reset_index(inplace=False))[['z','let', 'dose']]

In [ ]:
dfr=dfr.sort_values(by=['z', 'let','dose'])

In [ ]:
dfr.set_index(['z'],inplace=True) 

In [ ]:
for name, group in dfr.groupby('z'):

    dfr.loc[(name), "q"] = q.q.values[:n+1]


In [ ]:
for name, group in dfr.groupby('z'):
    DOSE=group.dose.values
    LET=group.let.values
    Q=group.q.values  
    dfr.loc[(name), "rbe"] = rbe_wedenberg(DOSE, LET, abx, Q)


In [ ]:
dfr.reset_index(inplace=True)

In [ ]:
dfr_tmp= pd.DataFrame()
dfr_tmp["mediana"]=dfr.groupby('z').rbe.median()
dfr_tmp["meana"]=dfr.groupby('z').rbe.mean()
dfr_tmp["q_down"]=dfr.groupby('z').rbe.quantile(0.075)
dfr_tmp["q_up"]=dfr.groupby('z').rbe.quantile(0.925)

# Biological Dose in SOBP

In [ ]:
limit=df_dose.z[df_dose.dose<=df_dose.dose.max()*0.01].iloc[0]

In [ ]:
figure2_df= pd.DataFrame()

In [ ]:
figure2_df["Distance_cm"] = rbe.z
figure2_df["Ext_Model_median"] = dfr.groupby('z').rbe.median().values*rbe.dose
figure2_df["Ext_Model_mean"] = dfr.groupby('z').rbe.mean().values*rbe.dose

figure2_df["Wedenberg"] = rbe.wed*rbe.dose
figure2_df["Wedenberg_quantile_down"] = rbe.wed_q_down*rbe.dose
figure2_df["Wedenberg_quantile_up"] = rbe.wed_q_up*rbe.dose

figure2_df["Physical__Dose"] = rbe.dose
figure2_df["Dose_for_RBE_1_1"] = rbe.dose*1.1
figure2_df["Ext_Model_quantile_down"] = dfr.groupby('z').rbe.quantile(0.075).values*rbe.dose
figure2_df["Ext_Model_quantile_up"] = dfr.groupby('z').rbe.quantile(0.925).values*rbe.dose

In [ ]:
fig,ax = plt.subplots(figsize=[14, 10])
matplotlib.rcParams.update({'font.size': 36})

ax2=ax.twinx()
ax.set_xlabel("Range [cm]",size=42, labelpad = 30)
ax.set_ylabel("Dose [Gy(RBE)]",size=42, labelpad = 30)
ax2.set_ylabel("\nLET "+r"$\;[keV \cdot \mu m^{-1}]$",size=30,rotation=90)

ax.set_xlim(0,limit)
ax.set_ylim(0,4)
ax.set_xticks(np.arange(0, 10, 1.0))

ax2.plot(rbe.z, rbe.let,'k.',markersize=1.2, label = "LET")

ax.plot(rbe.z, rbe.dose,c='k',label="Dose [Gy]")
ax.plot(figure2_df.Distance_cm,figure2_df.Ext_Model_mean,'g:',markersize=0.2, label= "our work: mean")
ax.plot(figure2_df.Distance_cm,figure2_df.Wedenberg,'r', label = "Wedenberg: mean")
ax.plot(figure2_df.Distance_cm, figure2_df.Dose_for_RBE_1_1,'b-.',markersize=0.1, label = "Dose for RBE 1.1") 

y1=figure2_df.Ext_Model_quantile_down
y2=figure2_df.Ext_Model_quantile_up
y3=figure2_df.Wedenberg_quantile_down
y4=figure2_df.Wedenberg_quantile_up

ax.fill_between(rbe.z,y1,y2, where=y2 >y1, facecolor='g', alpha=0.4, label = "our work: 85% CI")
ax.fill_between(rbe.z,y3,y4, where=y4 >y3, facecolor='r', alpha=0.4, label = "Wedenberg: 95% CI")

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')
ax.tick_params(axis='both', which='major', labelsize=28)

ax.legend(loc="center",bbox_to_anchor=(0.23, 0.77), borderaxespad=0.,fontsize=26)
ax2.legend(loc="center",bbox_to_anchor=(0.88, 0.95), borderaxespad=0.,fontsize=28)

# IN SOBP

In [ ]:
def dose_in_sobp(range_insobp):
    fig,ax = plt.subplots(figsize = (10,6))

    df_ = rbe_wedenberg(np.float64(rbe[rbe.z==range_insobp].dose.values), 
                        rbe[rbe.z == range_insobp].let.values, 
                        abx, q.q)*rbe[rbe.z == range_insobp].dose.values
    
    df_.hist(bins = 400, color='green',label = "praca Jeleń: rozkład",ax=ax,density=True)
    
    ax.axvline(figure2_df[figure2_df.Distance_cm == range_insobp].Ext_Model_median.values, 
                c='black',label='praca Jeleń: mediana')
    ax.axvspan(figure2_df[figure2_df.Distance_cm == range_insobp].Wedenberg_quantile_down.values, 
                figure2_df[figure2_df.Distance_cm == range_insobp].Wedenberg_quantile_up.values, 
                alpha=0.4, color='red',label="praca Wedenberg: 95% Przedział Ufności")
    
    ax.axvspan(figure2_df[figure2_df.Distance_cm == range_insobp].Ext_Model_quantile_down.values, 
                figure2_df[figure2_df.Distance_cm == range_insobp].Ext_Model_quantile_up.values, 
                alpha=0.4, color='green',label="praca Jeleń 85% Przedział Ufności")
    ax.axvline(figure2_df[figure2_df.Distance_cm == range_insobp].Dose_for_RBE_1_1.values, 
                c='blue',label='dawka dla RBE=1.1')
    ax.axvline(figure2_df[figure2_df.Distance_cm == range_insobp].Wedenberg.values, 
                c='red',label='praca Wedenberg: dla q=0.434')

    ax.set_xlabel("\nDawka Biologiczna [Gy(RBE)]")
    ax.set_ylabel("Rozkład gęstości \nprawdopodobieństwa [-]\n")
    ax.grid()
    ax.minorticks_on()
    ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')

In [ ]:
dose_in_sobp(7.015)

In [ ]:
dose_in_sobp(3.015)

In [ ]:
dose_in_sobp(8.515)